In [3]:
# Import necessary libraries
from GoogleNews import GoogleNews
import pandas as pd
import requests
from fake_useragent import UserAgent
import newspaper
from newspaper import fulltext
import re

In [4]:
# Define the keyword to search.
keyword = 'discapacidad'

In [5]:
# Perform news scraping from Google and extract the result into Pandas dataframe. 
googlenews = GoogleNews(lang='es', region='AR', period='1m', encode='utf-8')
googlenews.clear()
googlenews.search(keyword)
googlenews.get_page(1)
news_result = googlenews.result(sort=True)
news_data_df = pd.DataFrame.from_dict(news_result)

In [6]:
# Display information of dataframe.
news_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     20 non-null     object 
 1   media     20 non-null     object 
 2   date      20 non-null     object 
 3   datetime  0 non-null      float64
 4   desc      20 non-null     object 
 5   link      20 non-null     object 
 6   img       20 non-null     object 
dtypes: float64(1), object(6)
memory usage: 1.2+ KB


In [7]:
# Display header of dataframe.
news_data_df.head()

,title,media,date,datetime,desc,link,img
0,Un juez le ordenó al Gobierno restituir las pe...,La Gaceta,hace 7 minutos,NaN,La Justicia de Catamarca hizo lugar a la caute...,https://www.lagaceta.com.ar/nota/1102882/polit...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,Francos confirmó que el Gobierno promulgará la...,El Liberal,hace 12 minutos,NaN,"El jefe de Gabinete, Guillermo Francos, confir...",https://www.elliberal.com.ar/nota/58661/2025/0...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,"El Gobierno promulgará la ley de discapacidad,...",Diario Río Negro,hace 16 minutos,NaN,"La decisión la confirmó el jefe de Gabinete, G...",https://www.rionegro.com.ar/politica/el-gobier...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,Jornada formativa organizada por el ayuntamien...,Onda Cero,hace 17 minutos,NaN,Fernando Romero director del área de formación...,https://www.ondacero.es/emisoras/comunidad-val...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,El Gobierno anunció que promulgará la Ley de D...,Diario El Día de La Plata,hace 23 minutos,NaN,"El jefe de Gabinete, Guillermo Francos, dijo q...",https://www.eldia.com/nota/2025-9-15-10-22-0-e...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [10]:
# ua = UserAgent()
# news_data_df_with_text = []
# for index, headers in news_data_df.iterrows():
#     news_title = str(headers['title'])
#     news_media = str(headers['media'])
#     news_update = str(headers['date'])
#     news_timestamp = str(headers['datetime'])
#     news_description = str(headers['desc'])
#     news_link = str(headers['link'])
#     print(news_link)
#     news_img = str(headers['img'])
#     try:
#         # html = requests.get(news_link).text
#         html = requests.get(news_link, headers={'User-Agent':ua.chrome}, timeout=15).text
#         text = fulltext(html)
#         print('Text Content Scraped')
#     except:
#         print('Text Content Scraped Error, Skipped')
#         pass
#     news_data_df_with_text.append([news_title, news_media, news_update, news_timestamp, 
#                                          news_description, news_link, news_img, text])

# news_data_with_text_df = pd.DataFrame(news_data_df_with_text, columns=['Title', 'Media', 'Update', 'Timestamp',
#                                                                     'Description', 'Link', 'Image', 'Text'])

from newspaper import Article

ua = UserAgent()
news_data_df_with_text = []

for index, headers in news_data_df.iterrows():
    news_title = str(headers['title'])
    news_media = str(headers['media'])
    news_update = str(headers['date'])
    news_timestamp = str(headers['datetime'])
    news_description = str(headers['desc'])
    news_link = str(headers['link'])
    news_img = str(headers['img'])

    print(news_link)

    text = ""  # initialize

    # First try Newspaper (more robust)
    try:
        article = Article(news_link, browser_user_agent=ua.chrome)
        article.download()
        article.parse()
        text = article.text
        print('Text Content via newspaper3k')
    except Exception as e:
        print(f'Newspaper extraction failed: {e}')
        # fallback to requests + fulltext
        try:
            html = requests.get(news_link, headers={'User-Agent': ua.chrome}, timeout=5).text
            text = fulltext(html)
            print('Text Content via fulltext fallback')
        except Exception as e2:
            print(f'Fallback fulltext failed: {e2}')
            # text remains empty

    news_data_df_with_text.append([
        news_title, news_media, news_update, news_timestamp,
        news_description, news_link, news_img, text
    ])

news_data_with_text_df = pd.DataFrame(
    news_data_df_with_text,
    columns=['Title', 'Media', 'Update', 'Timestamp',
             'Description', 'Link', 'Image', 'Text']
)


https://www.lagaceta.com.ar/nota/1102882/politica/juez-le-ordeno-al-gobierno-restituir-pensiones-discapacidad-catamarca.html&ved=2ahUKEwiF3rLP9tqPAxUeIbkGHcZHOzMQxfQBegQICRAC&usg=AOvVaw13ocLehvn9hR1icZXH3q2H
Text Content via newspaper3k
https://www.elliberal.com.ar/nota/58661/2025/09/francos-confirmo-que-el-gobierno-promulgara-la-ley-de-discapacidad-pero-no-la-aplicara-hasta-definir-el-financiamiento&ved=2ahUKEwiF3rLP9tqPAxUeIbkGHcZHOzMQxfQBegQIBxAC&usg=AOvVaw2Lc08PSIRqINVC6vMN8a9G
Text Content via newspaper3k
https://www.rionegro.com.ar/politica/el-gobierno-promulgara-la-ley-de-discapacidad-descartan-su-aplicacion-hasta-definir-como-sera-financiada-4299305/&ved=2ahUKEwiF3rLP9tqPAxUeIbkGHcZHOzMQxfQBegQIChAC&usg=AOvVaw3Z5vb23jzOWL-lizJj5EaN
Newspaper extraction failed: Article `download()` failed with 404 Client Error: Not Found for url: https://www.rionegro.com.ar/politica/el-gobierno-promulgara-la-ley-de-discapacidad-descartan-su-aplicacion-hasta-definir-como-sera-financiada-4299305/&

In [11]:
# Display the entire dataframe for sample checking.
news_data_with_text_df

,Title,Media,Update,Timestamp,Description,Link,Image,Text
0,Un juez le ordenó al Gobierno restituir las pe...,La Gaceta,hace 7 minutos,nan,La Justicia de Catamarca hizo lugar a la caute...,https://www.lagaceta.com.ar/nota/1102882/polit...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","En su resolución, el juez indica que la Andis ..."
1,Francos confirmó que el Gobierno promulgará la...,El Liberal,hace 12 minutos,nan,"El jefe de Gabinete, Guillermo Francos, confir...",https://www.elliberal.com.ar/nota/58661/2025/0...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","El jefe de Gabinete, Guillermo Francos, confir..."
2,"El Gobierno promulgará la ley de discapacidad,...",Diario Río Negro,hace 16 minutos,nan,"La decisión la confirmó el jefe de Gabinete, G...",https://www.rionegro.com.ar/politica/el-gobier...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
3,Jornada formativa organizada por el ayuntamien...,Onda Cero,hace 17 minutos,nan,Fernando Romero director del área de formación...,https://www.ondacero.es/emisoras/comunidad-val...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",JavaScript is disabled in your browser. Please...
4,El Gobierno anunció que promulgará la Ley de D...,Diario El Día de La Plata,hace 23 minutos,nan,"El jefe de Gabinete, Guillermo Francos, dijo q...",https://www.eldia.com/nota/2025-9-15-10-22-0-e...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","Diario El Día de La Plata, fundado el 2 de Mar..."
5,El Gobierno promulgará la Ley de Discapacidad ...,Diario Mendoza,hace 25 minutos,nan,El Gobierno anunció que promulgará la Ley de D...,https://www.diariomendoza.com.ar/politica/el-g...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
6,"El Gobierno promulgará la Ley de Discapacidad,...",Cadena 3 Argentina,hace 25 minutos,nan,El Gobierno anunció la promulgación de la Ley ...,https://www.cadena3.com/noticia/politica-y-eco...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",El Gobierno anunció la promulgación de la Ley ...
7,Emergencia en Discapacidad: el Gobierno la pro...,Página | 12,hace 25 minutos,nan,El jefe de Gabinete sostuvo que su intención e...,https://www.pagina12.com.ar/857939-emergencia-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Jaqueado por el frente social y político que s...
8,Ley de discapacidad: la Casa Rosada cede y la ...,Red43,hace 28 minutos,nan,El Gobierno nacional confirmó que promulgará l...,https://www.red43.com.ar/nota/2025-9-15-10-29-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
9,"El Gobierno promulgará la Ley de Discapacidad,...",El Eco,hace 29 minutos,nan,"El jefe de Gabinete, Guillermo Francos, confir...",https://www.eleco.com.ar/nacionales/el-gobiern...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",¡Ups! Esta página no pudo ser reconocida por n...


In [12]:
# Drops missiong or empty text
filtered_df = news_data_with_text_df[
    news_data_with_text_df['Text'].notna() &
    (news_data_with_text_df['Text'].str.strip() != "")
]


In [13]:
# Save the result dataframe into a CSV file.
filtered_df.to_csv("./data/news_data_with_text.csv")

In [14]:
# Reload the saved news data content from a CSV file.
news_data_with_text_df = pd.read_csv("./data/news_data_with_text.csv",  index_col=0)

In [15]:
# Select the top 10 news records (the latest 10) for summarization.
news_data_with_text_df = news_data_with_text_df.head(10)

In [16]:
# Display the scraped text content from the first news record.
print(news_data_with_text_df['Text'].values[0])

En su resolución, el juez indica que la Andis debe "en el plazo de veinticuatro horas desde el dictado de la presente, restablezca la totalidad de las pensiones a los titulares y pague el importe de los haberes de pensión retenidos a la fecha".


In [17]:
# Count the number of words in the first news record.
len(re. findall(r'\w+', news_data_with_text_df['Text'].values[0]))

44

In [18]:
# Concatenate all the top 10 news record's text content into a single string.
news_text_content_string = news_data_with_text_df.to_string(columns=['Text'], header=False, index=False)
print(news_text_content_string)

In [19]:
# Count the number of words in the top 10 news record's text content.
len(re. findall(r'\w+', news_text_content_string))

2151

In [21]:
# Import necessary libraries
import openai
import platform
import os

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anapa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [22]:
# Define the function to count the number of tokens.
def count_tokens(stringname):
    tokens = word_tokenize(stringname)
    return len(tokens)

In [26]:
# Display the number of tokens from the top 10 news record's text content.
stringname = news_text_content_string

token_count = count_tokens(stringname)
print(f"Number of tokens: {token_count}")

Number of tokens: 2310


In [27]:
# Define the function to divide the top 10 news record's text content into segments of 2000 tokens 
# with the overlap of 100 tokens to avoid losing information from the split.
def break_up_file(tokens, chunk_size, overlap_size):
    if len(tokens) <= chunk_size:
        yield tokens
    else:
        chunk = tokens[:chunk_size]
        yield chunk
        yield from break_up_file(tokens[chunk_size-overlap_size:], chunk_size, overlap_size)

def break_up_file_to_chunks(stringname, chunk_size=2000, overlap_size=100):
    tokens = word_tokenize(stringname)
    return list(break_up_file(tokens, chunk_size, overlap_size))

In [28]:
# Trial run of dividing function.
stringname = news_text_content_string

chunks = break_up_file_to_chunks(stringname)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} tokens")

Chunk 0: 2000 tokens
Chunk 1: 410 tokens


In [29]:
# Define the function to convert tokenized text back to normal text prompts.
def convert_to_detokenized_text(tokenized_text):
    prompt_text = " ".join(tokenized_text)
    prompt_text = prompt_text.replace(" 's", "'s")
    return prompt_text

In [30]:
# Configure the baseline configuration of the OpenAI library.
openai.api_type = "azure"
openai.api_base = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"

In [31]:
# Perform news text content summarization by Azure OpenAI Service (GPT3) for each chunk.
stringname = news_text_content_string

prompt_response = []
chunks = break_up_file_to_chunks(stringname)

for i, chunk in enumerate(chunks):
    print("Processing chunk " + str(i))
    prompt_request = "Summarize this news content: " + convert_to_detokenized_text(chunks[i])
    response = openai.Completion.create(
            engine="eason-text-davinci-002",
            prompt=prompt_request,
            temperature=.5, # Default is 1.
            max_tokens=500,
            top_p=1 # Default is 0.5.
    )
    
    prompt_response.append(response["choices"][0]["text"].strip())

Processing chunk 0


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [25]:
# Define the prompt to perform summarization into 1,500 words for each summarized content.
prompt_request = "Consolidate these news content summaries into 1500 words sentences: " + str(prompt_response)

In [26]:
# Perform summarization by Azure OpenAI Service (GPT3) for each chunk of summarized content.
response = openai.Completion.create(
        engine="PLEASE_ENTER_YOUR_AOAI_MODEL_DEPLOYMENT_NAME",
        prompt=prompt_request,
        temperature=.5, # Default is 1.
        max_tokens=1000,
        top_p=1 # Default is 0.5.
    )

In [28]:
# Display the final summary from the top 10 news record's text content.
news_content_summary = response["choices"][0]["text"].strip()
print(news_content_summary)

The amount of text available on the internet is growing at an exponential rate. A large portion of this text is in the form of news content. As the amount of news content grows, so does the need for automated methods of summarizing this content.

One such method is to use large language models, or LLMs. LLMs are a type of generative AI that are becoming increasingly popular. These models are able to generate text, songs, poems, essays, and even pass exams. However, there is deep concern within the tech industry that these models are being developed too quickly and that they may someday run out of control and damage or destroy human society.

This concern was renewed when OpenAI, a research lab founded by student Jack Hughes, released the ChatGPT chatbot. This chatbot is able to generate responses to questions that are similar to those of a human. The release of this chatbot has caused a lot of concern within the tech industry, with many people calling for a moratorium on the developmen

In [37]:
# Select the necessary columns and convert the dataframe into HTML for showing in the email.
news_data_with_text_df_1 = news_data_with_text_df[["Title", "Media", "Timestamp", "Description", "Link"]]
news_data_with_text_df_1_html = news_data_with_text_df_1.to_html(index=False)

In [43]:
# Email the GPT news summary with the news source reference table via MailJet. 
from mailjet_rest import Client
import os
api_key = 'PLEASE_ENTER_YOUR_OWNED_MAILJET_API_KEY_NAME'
api_secret = 'PLEASE_ENTER_YOUR_OWNED_MAILJET_API_KEY_SECRET'
mailjet = Client(auth=(api_key, api_secret), version='v3.1')
data = {
  'Messages': [
    {
      "From": {
        "Email": "easonlai888@gmail.com",
        "Name": "Eason"
      },
      "To": [
        {
          "Email": "easonlai888@gmail.com",
          "Name": "Eason"
        }
      ],
      "Subject": "GPT News Summary of Today",
      "HTMLPart": "<h3>Here is the news summary of GPT for today.</h3>{}<br><br> \
                   <h3>GPT News Summary Sources</h3>{}" \
                   .format(news_content_summary, news_data_with_text_df_1_html),
    }
  ]
}
result = mailjet.send.create(data=data)

print(result.status_code)
print(result.json())


200
{'Messages': [{'Status': 'success', 'CustomID': '', 'To': [{'Email': 'easonlai888@gmail.com', 'MessageUUID': '3254b42f-5509-47d5-b473-a9bfdd8a1511', 'MessageID': 576460769578586240, 'MessageHref': 'https://api.mailjet.com/v3/REST/message/576460769578586240'}], 'Cc': [], 'Bcc': []}]}
